## Web scraping exercise

Define a generic function `SOS_help` which retrieves help results from Stack Overflow Stunning results. <br>

The following command works just fine:

In [3]:
import numpy as np
import pandas as pd
import requests
import traceback
np.random.uniform(-1, 1, size=100)

array([-0.40773625,  0.81176446,  0.72645119, -0.0459896 , -0.40666746,
       -0.99381218, -0.91939381,  0.77033487,  0.80817648, -0.32997464,
       -0.70024738, -0.13368828,  0.66374592, -0.4276838 ,  0.67467887,
       -0.91260591,  0.48044207, -0.4919603 ,  0.12043443, -0.66953613,
        0.95478493, -0.00523688,  0.82214906,  0.42209764,  0.70103916,
       -0.48720515,  0.83457048, -0.27932136,  0.31832036,  0.53427355,
        0.24421323,  0.04629889,  0.6294784 , -0.6083971 ,  0.24484633,
        0.98482922, -0.21221029,  0.84640803, -0.36696821, -0.00660736,
        0.50102283, -0.94632513, -0.70377903,  0.04360287,  0.35855261,
       -0.49263054,  0.75692524,  0.16210997,  0.40933262, -0.65074272,
        0.35801824,  0.40172286,  0.4840267 , -0.05141395, -0.47535601,
       -0.54178091,  0.9895423 ,  0.35309066, -0.72833947,  0.05938232,
       -0.5552884 , -0.1569402 ,  0.3285149 , -0.24700688,  0.39813186,
       -0.92633859,  0.16442011, -0.17216845, -0.24513454,  0.36

`print_out` prints command execution

In [4]:
def print_output(command):
    command = 'print(' + command + ')'
    return exec(command)

In [5]:
print_output(command = 'np.random.uniform(-1, 1, size=100)')

[ 0.15290593 -0.5485631   0.14909195 -0.39235759 -0.72940938 -0.68302318
  0.83195962  0.4352119  -0.25402972  0.06554824 -0.43369561 -0.8548954
 -0.86131841  0.00646655 -0.17618237 -0.9194963  -0.46972425 -0.89282944
 -0.78689991  0.60195441 -0.26932346 -0.02578805 -0.42556805  0.75869001
 -0.08452757  0.52378133 -0.50808762  0.13240001 -0.43390195  0.7588319
  0.2400353  -0.45800304  0.07995439  0.88990835 -0.12362981  0.02984394
 -0.96926338  0.99519036 -0.3710208   0.03478207  0.95038194 -0.16411235
  0.89255754 -0.95280733  0.67039594  0.26115719  0.43415585  0.48587332
  0.46692522 -0.97203184 -0.95785736  0.58645067 -0.66266431  0.14403065
 -0.88428984  0.15344608  0.6069378   0.988799   -0.68714043 -0.41797393
 -0.89214938 -0.2133785  -0.30428374 -0.64256044 -0.74432015 -0.92587704
  0.05363466 -0.86502809 -0.68878371 -0.47564837 -0.88610598  0.35331937
 -0.20261093 -0.78413648  0.12680174  0.11888673 -0.16446678 -0.27796811
  0.7131275   0.73917536 -0.91070163  0.02655728 -0.0

if `print_out` results in an error, this is printed

In [5]:
print_output(command = 'nn.random.uniform(-1, 1, size=100)')

NameError: name 'nn' is not defined

In [6]:
print_output(command = 'np.random.uniform(-1, 1, siz=100)')

TypeError: uniform() got an unexpected keyword argument 'siz'

Create a function `get_SOS_help` which: <br>
    - Prints "works as intended" if no error. <br>
    - Prints the first link from stack overflow related to the error. As an example: <br>
        `print_output(command = 'np.random.uniform(-1, 1, siz=100)'`
        should retrieve the following link:
        https://stackoverflow.com/questions/72537485/typeerror-uniform-got-an-unexpected-keyword-argument-low-size <br>
    - Prints the most voted help
    - Opens a new browser using the link

## Create a malfunctioning code and use this function on it

In [6]:
import os
from openai import OpenAI
from dotenv import load_dotenv
import traceback
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Cargar la clave de API desde el archivo .env
load_dotenv()
api_key = os.getenv("OPENAI_KEY")\


# Configurar el cliente de OpenAI
client = OpenAI(api_key=api_key)

def get_SOS_help(error_message):
    url = f"https://www.google.com/search?q={error_message} site:stackoverflow.com"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36"
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
    except requests.RequestException as e:
        print("❌ Error al conectar con Google:", e)
        return None
    
    allData = soup.find_all("div", {"class": "g"})
    urls = []
    
    for data in allData:
        link_tag = data.find("a")
        if link_tag and link_tag.get('href'):
            urls.append(link_tag['href'])
    
    if not urls:
        print("❌ No se encontraron resultados en StackOverflow")
        return None
    
    print("🔍 Los mejores resultados de StackOverflow son:")
    print(f"👉 {urls[0]}")
    
    try:
        response = requests.get(urls[0], headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
    except requests.RequestException as e:
        print("❌ Error al conectar con StackOverflow:", e)
        return None

    votes = soup.find_all("div", {"class": "js-vote-count"})
    comments = soup.find_all("div", {"class": "s-prose js-post-body"})
    
    votes_list = [int(vote.get_text().strip()) for vote in votes if vote.get_text().strip().isdigit()]
    comments_list = [comment.get_text().strip() for comment in comments]
    
    if not votes_list or not comments_list:
        print("⚠️ No se pudieron extraer comentarios o votos de la respuesta.")
        return None
    
    data = list(zip(votes_list, comments_list))
    df = pd.DataFrame(data, columns=['Votes', 'Comment'])
    df = df.sort_values(by='Votes', ascending=False)
    
    top_comment = df.Comment.iloc[0]
    print("\n🎉 ¡Aquí tienes el comentario más votado! 🎉\n")
    
    return top_comment

def get_AI_help(error_message):
    # Construir el prompt con instrucciones específicas
    prompt = (
        f"Eres un asistente que ayuda a programadores con errores de Python. "
        f"Por favor, proporciona una explicación detallada y una solución para el error: '{error_message}'. "
        "Usa el siguiente formato:\n\n"
        "⚠️ **Error encontrado:** [Explicación del error]\n\n"
        "💡 **Posible solución:** [Solución propuesta con ejemplo de código]\n\n"
        "Incluye ejemplos de código y traceback cuando sea relevante."
    )

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=500,
            temperature=0.7
        )
        
        # Extraer y devolver la respuesta formateada de la IA
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error al generar el comentario con OpenAI: {e}")
        return None

def execute_code_with_help(command):
    try:
        exec(command)
    except Exception as e:
        error_message = str(e)
        full_traceback = traceback.format_exc()
        print(f"⚠️ Error encontrado: {error_message}")
        
        # Buscar ayuda en StackOverflow
        print("🔍 Buscando solución en StackOverflow...")
        solution = get_SOS_help(error_message)

        if solution:
            print("💡 Posible solución encontrada en StackOverflow:")
            print(solution)
        else:
            print("❌ No se encontró una solución relevante en StackOverflow.")
        
        # Consultar a OpenAI para obtener una respuesta formateada
        print("\n🤖 Consultando a GPT-3.5 para obtener ayuda...\n")
        ai_solution = get_AI_help(error_message)

        if ai_solution:
            print(ai_solution)
        else:
            print("❌ No se pudo obtener una respuesta de GPT-3.5.")


In [7]:
execute_code_with_help('int("hola")')


⚠️ Error encontrado: invalid literal for int() with base 10: 'hola'
🔍 Buscando solución en StackOverflow...
🔍 Los mejores resultados de StackOverflow son:
👉 https://stackoverflow.com/questions/1841565/valueerror-invalid-literal-for-int-with-base-10

🎉 ¡Aquí tienes el comentario más votado! 🎉

💡 Posible solución encontrada en StackOverflow:
The error message means that the string provided to int could not be parsed as an integer. The part at the end, after the :, shows the string that was provided.
In the case described in the question, the input was an empty string, written as ''.
Here is another example - a string that represents a floating-point value cannot be converted directly with int:
>>> int('55063.000000')
Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
ValueError: invalid literal for int() with base 10: '55063.000000'

Instead, convert to float first:
>>> int(float('55063.000000'))
55063

🤖 Consultando a GPT-3.5 para obtener ayuda...

⚠️ **Error encon